In [144]:
# import libraries
from rdflib import Graph,Literal,RDF,URIRef
from rdflib.namespace import FOAF,XSD,RDFS
from rdflib import Namespace
import pandas as pd
import string
import random
import numpy as np

In [108]:
g = Graph()

In [109]:
KG_SDM = Namespace("http://kg_sdm.org/")
g.bind("kg_sdm",KG_SDM) # the uri will be repliced

In [110]:
# Author subclass Person
# Author: school
# Person: name
# Create a table of names and schools which will be later sliced to Author, Editor etc.

# adding Persons and subclasses

In [111]:
people_names = pd.read_csv('data_kg_sdm/authors.csv')
# people_names = people_names.drop(columns='ID')
people_names.rename(columns = {'name':'person_name'}, inplace=True)
people_names.head()

,ID,person_name
0,57218202833,Gautam A.
1,7004904337,Crandall J.W.
2,7005513246,Goodrich M.A.
3,6508306234,de Moura Oliveira P.B.
4,9277159100,Hedengren J.D.


In [112]:
schools = pd.read_csv('data_kg_sdm/schools.csv')
# schools = schools.drop(columns='ID')
schools.rename(columns = {'name':'school_name'}, inplace=True)
schools.head()

,ID,school_name
0,1,"Computer Science Department, Brigham Young Uni..."
1,2,"INESC-TEC Technology and Science, Campus da FE..."
2,3,"Department of Engineering, University of Trás..."
3,4,"Department of Chemical Engineering, Brigham Y..."
4,5,"Task Committee Secretary and Lead Engineer, Op..."


In [113]:
academics = people_names.loc[:schools.size-1]
academics['school_name'] = schools.school_name
academics.head()

/tmp/ipykernel_40793/3816719288.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  academics['school_name'] = schools.school_name


,ID,person_name,school_name
0,57218202833,Gautam A.,"Computer Science Department, Brigham Young Uni..."
1,7004904337,Crandall J.W.,"INESC-TEC Technology and Science, Campus da FE..."
2,7005513246,Goodrich M.A.,"Department of Engineering, University of Trás..."
3,6508306234,de Moura Oliveira P.B.,"Department of Chemical Engineering, Brigham Y..."
4,9277159100,Hedengren J.D.,"Task Committee Secretary and Lead Engineer, Op..."


In [114]:
# We suppose 60% are Authors, 20% Reviewsrs, 10% Chair or Editor

In [115]:
total = len(academics)
nb_auth = int(total * 0.6)
nb_rev = int(total * 0.2)
nb_chair_editor = int(total * 0.1)

In [116]:
authors = academics.loc[:nb_auth]
reviewers = academics.loc[nb_auth : nb_auth + nb_rev]
chair = academics.loc[nb_auth + nb_rev : nb_auth + nb_rev + nb_chair_editor ]
editor = academics.loc[ nb_auth + nb_rev + nb_chair_editor :nb_auth + nb_rev + nb_chair_editor + nb_chair_editor]

In [117]:
authors.head()

,ID,person_name,school_name
0,57218202833,Gautam A.,"Computer Science Department, Brigham Young Uni..."
1,7004904337,Crandall J.W.,"INESC-TEC Technology and Science, Campus da FE..."
2,7005513246,Goodrich M.A.,"Department of Engineering, University of Trás..."
3,6508306234,de Moura Oliveira P.B.,"Department of Chemical Engineering, Brigham Y..."
4,9277159100,Hedengren J.D.,"Task Committee Secretary and Lead Engineer, Op..."


In [118]:


def URLparse(url:str):
    url=url.replace("\'","_").replace("\"","_")
    for i in string.punctuation:
        url = url.replace(i,"_")
    url = url.replace(" ","_")
    return url



In [119]:
person_type_list = [authors,reviewers,chair,editor]
person_sub_type_ls = ["Author","Reviewers","Chair","Editor"]

In [120]:


Person = URIRef("http://kg_sdm.org/Person")

for i,person_tp in enumerate(person_type_list):
    preson_sub_type = URIRef(f"http://kg_sdm.org/{person_sub_type_ls[i]}")
    
    for name, school in zip(person_tp['person_name'], person_tp['school_name']):
    #   editor_node = URIRef(f"http://kg_sdm.org/{URLparse(name)}")
        parsed_name = URLparse(name)
        person_node = URIRef(f"http://kg_sdm.org/Person/{parsed_name}")
        name_lit = Literal(str(name))
        school_lit = Literal(str(school))

        # add subclass type
        g.add((person_node, RDF.type, preson_sub_type))
        # add school
        g.add((person_node, KG_SDM.school, school_lit))
        # add name of person
        g.add((person_node, FOAF.name, name_lit))



In [121]:
g.serialize()

'@prefix foaf: <http://xmlns.com/foaf/0.1/> .\n@prefix kg_sdm: <http://kg_sdm.org/> .\n\n<http://kg_sdm.org/Person/Aarsnes_U_J_> a kg_sdm:Author ;\n    kg_sdm:school " Cardiff University Brain Research Imaging Centre (CUBRIC), Cardiff University, Cardiff, United Kingdom" ;\n    foaf:name "Aarsnes U.J." .\n\n<http://kg_sdm.org/Person/Abaimov_S_> a kg_sdm:Author ;\n    kg_sdm:school " Department of Biology, Utah Valley University, SB 242M, Orem, UT  84058, United States" ;\n    foaf:name "Abaimov S." .\n\n<http://kg_sdm.org/Person/Abildskov_T_J_> a kg_sdm:Reviewers ;\n    kg_sdm:school " Department of Physics and Astronomy, KU Leuven, Celestijnenlaan 200D, Heverlee, B-3001, Belgium" ;\n    foaf:name "Abildskov T.J." .\n\n<http://kg_sdm.org/Person/Abliz_D_> a kg_sdm:Author ;\n    kg_sdm:school " Department of Instructional Psychology and Technology, Brigham Young University, 150-H MCKB, Provo, UT  84602, United States" ;\n    foaf:name "Abliz D." .\n\n<http://kg_sdm.org/Person/Abraham_Mar

# Submission

In [169]:
author_article = pd.read_csv('data_kg_sdm/author_written_article.csv')
author_article.dropna(inplace=True)
author_article['author_ID'] = author_article['author_ID'].astype(int)
author_article.head()

,author_ID,article_ID
0,57218202833,1
1,7004904337,1
2,7005513246,1
3,6508306234,2
4,9277159100,2


In [170]:
articles = pd.read_csv('data_kg_sdm/articles.csv')

In [171]:
paper_type = ['Demo','FullPaper', 'Poster', 'Short']
keywords = ['ML', 'NLP', 'Database', 'Graph']

In [172]:
articles['year'] = [ random.randint(2000,2022) for i in range(len(articles))]
articles['type'] = [random.choice(paper_type) for i in range(len(articles))]
articles['keyword'] = [random.choice(keywords) for i in range(len(articles))]
articles['accepted'] = ""
articles['accepted'].loc[:500] = True
articles['accepted'].loc[500:] = False

/tmp/ipykernel_40793/3662288323.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  articles['accepted'].loc[:500] = True
/tmp/ipykernel_40793/3662288323.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  articles['accepted'].loc[500:] = False


In [173]:
articles.head()

,ID,title,volume,DOI,year,type,keyword,accepted
0,1,Self-assessment of Proficiency of Intelligent ...,1210 AISC,10.1007/978-3-030-51758-8_15,2015,Poster,Graph,True
1,2,Bridging theory to practice: Feedforward and c...,695 LNEE,10.1007/978-3-030-58653-9_3,2020,FullPaper,Graph,True
2,3,Development of sediment management guidelines ...,146,10.1061/(ASCE)HY.1943-7900.0001822,2009,FullPaper,Graph,True
3,4,Structural design space exploration using prin...,20,10.1115/1.4047428,2009,Poster,NLP,True
4,5,Religion-focused dating apps: A Q methodology ...,55,10.1016/j.tele.2020.101448,2003,Short,NLP,True


In [174]:
authors

,ID,person_name,school_name
0,57218202833,Gautam A.,"Computer Science Department, Brigham Young Uni..."
1,7004904337,Crandall J.W.,"INESC-TEC Technology and Science, Campus da FE..."
2,7005513246,Goodrich M.A.,"Department of Engineering, University of Trás..."
3,6508306234,de Moura Oliveira P.B.,"Department of Chemical Engineering, Brigham Y..."
4,9277159100,Hedengren J.D.,"Task Committee Secretary and Lead Engineer, Op..."
...,...,...,...
1299,56701050600,Reynolds P.R.,"Brigham Young University, 350 CB, Provo, UT 8..."
1300,57203184524,Cramer J.,"Dept. of Civil and Environmental Engineering, ..."
1301,57201465968,Adams D.,Dept. of Agricultural and Biochemical Enginee...
1302,56248395300,Brown T.,School of Sustainable Engineering and the Bui...


In [175]:
article_pub

,ID,title,volume,DOI,year,type,keyword,accepted,author_ID,article_ID
0,1,Self-assessment of Proficiency of Intelligent ...,1210 AISC,10.1007/978-3-030-51758-8_15,2005,Poster,Database,True,57218202833,1
1,1,Self-assessment of Proficiency of Intelligent ...,1210 AISC,10.1007/978-3-030-51758-8_15,2005,Poster,Database,True,7004904337,1
2,1,Self-assessment of Proficiency of Intelligent ...,1210 AISC,10.1007/978-3-030-51758-8_15,2005,Poster,Database,True,7005513246,1
3,2,Bridging theory to practice: Feedforward and c...,695 LNEE,10.1007/978-3-030-58653-9_3,2021,Poster,Database,True,6508306234,2
4,2,Bridging theory to practice: Feedforward and c...,695 LNEE,10.1007/978-3-030-58653-9_3,2021,Poster,Database,True,9277159100,2
...,...,...,...,...,...,...,...,...,...,...
3935,880,Dryland agriculture in north america,NaN,10.1007/978-3-319-47928-6_15,2021,Short,ML,False,7401698226,880
3936,880,Dryland agriculture in north america,NaN,10.1007/978-3-319-47928-6_15,2021,Short,ML,False,8702836000,880
3937,880,Dryland agriculture in north america,NaN,10.1007/978-3-319-47928-6_15,2021,Short,ML,False,7005306331,880
3938,880,Dryland agriculture in north america,NaN,10.1007/978-3-319-47928-6_15,2021,Short,ML,False,7102554886,880


#### join articles and authors

In [176]:
article_pub = pd.merge(articles, author_article, left_on='ID', right_on='article_ID')
articles_publishedin = pd.merge(article_pub, authors, left_on='author_ID', right_on='ID',how='left')
articles_publishedin = articles_publishedin[['title', 'year', 'type', 'keyword', 'accepted', 'person_name', 'school_name']]
articles_publishedin.dropna(inplace=True)

# TODO: ACCEPTED

In [177]:
len(articles_publishedin)

2879

In [178]:
for a in articles.itertuples():
    print(a)
    break

Pandas(Index=0, ID=1, title='Self-assessment of Proficiency of Intelligent Systems: Challenges and Opportunities', volume='1210 AISC', DOI='10.1007/978-3-030-51758-8_15', year=2015, type='Poster', keyword='Graph', accepted=True)


In [179]:
for _, article_title, year, paper_type, keyword, accepted, author_name, school_name in articles_publishedin.itertuples():
    # get the author node
    author_node = URIRef(f"http://kg_sdm.org/Person/{URLparse(author_name)}")
    
    # create the submission onde
    submission_node = URIRef(f"http://kg_sdm.org/Submission/{URLparse(article_title)}")
    g.add((submission_node, RDF.type, KG_SDM.Submission))
    
    # author wrote a paper
    g.add((author_node, KG_SDM.writes, submission_node))
                            
    
    # data for submission
    paper_title_lit = Literal(str(article_title))
    paper_year_lit = Literal(int(year))
    g.add((submission_node, KG_SDM.paper_title, paper_title_lit))
    g.add((submission_node, KG_SDM.paper_year, paper_year_lit))
                            
    # keywords
    # create the node
    # NLP rdf type Keyword
    keyword_node = URIRef(f"http://kg_sdm.org/{keyword}")
#     g.add((keyword_node, RDF.type, KG_SDM.Keyword))
    # connect with submission
    # Submisson a NLP
    g.add((submission_node, KG_SDM.related_to, keyword_node))

    # paper type
    # create paper type
    paper_node = URIRef(f"http://kg_sdm.org/{URLparse(paper_type)}")
#     g.add((paper_node, RDF.type, KG_SDM.PaperType))
    g.add((submission_node, KG_SDM.of_type, paper_node))
    
    
    

# ReviewProcess

In [187]:
reviewers.head()

,ID,person_name,school_name
1303,57203180153,Misha R.,"Brigham Young University, 350 CB, Provo, UT 84..."
1304,7005486887,Sachdev A.,"Fire Science and Technology Department, Sandi..."
1305,34874038200,Wright G.A.,"School of Mechanical Engineering, University o..."
1306,57201366662,Collins N.S.,"Manufacturing Engineering Technology Program,..."
1307,57201358872,Farnsworth M.S.,"Environment Materials and Components Center, ..."


In [188]:
reviewProcess = articles_publishedin.drop_duplicates(subset=['title'])

In [189]:
reviewProcess.head()

,title,year,type,keyword,accepted,person_name,school_name
0,Self-assessment of Proficiency of Intelligent ...,2015,Poster,Graph,True,Gautam A.,"Computer Science Department, Brigham Young Uni..."
3,Bridging theory to practice: Feedforward and c...,2020,FullPaper,Graph,True,de Moura Oliveira P.B.,"Department of Chemical Engineering, Brigham Y..."
6,Development of sediment management guidelines ...,2009,FullPaper,Graph,True,Hajimirzaie S.M.,"Department of Mechanical Engineering, Brigham ..."
8,Structural design space exploration using prin...,2009,Poster,NLP,True,Bunnell S.,"School of Communications, Brigham Young Univer..."
13,Religion-focused dating apps: A Q methodology ...,2003,Short,NLP,True,Richardson M.,"North Carolina State University, Department of..."


In [190]:
list(reviewers.sample(random.randint(2,4))['person_name'].values)

['Clark M.R.', 'Poon T.C.', 'Bertrand R.']

In [191]:
reviews = []
comments = []
rejacc = []

# for i in range(len(decision)):
#     c=[]
#     l=[]
#     for j in range(randint(2,4)):
#         r.append(''.join(choice(string.ascii_letters))
#     c.append(reviewers.sample(len(r))['person_name'])
#     print(c,r)

for i in range(len(reviewProcess)): # for every row
    # create N reviewers and comments
    N = random.randint(2,4)
    rev_list = list(reviewers.sample(N)['person_name'])
    acc = [random.random()>0.5 for i in range(N)]
    com_list = []
    for j in range(N):
        comment = ''.join((random.choice('abcdefghijklmnopqrstuvwxyz') for i in range(5)))
        com_list.append(comment)
    reviews.append(rev_list)
    comments.append(com_list)
    rejacc.append(acc)
    
reviewProcess['reviewer_name'] = reviews
reviewProcess['comment'] = comments
reviewProcess['decision'] = rejacc

/tmp/ipykernel_40793/1823661669.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviewProcess['reviewer_name'] = reviews
/tmp/ipykernel_40793/1823661669.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviewProcess['comment'] = comments
/tmp/ipykernel_40793/1823661669.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

In [ ]:
rev

In [192]:
reviewProcess.head()

,title,year,type,keyword,accepted,person_name,school_name,reviewer_name,comment,decision
0,Self-assessment of Proficiency of Intelligent ...,2015,Poster,Graph,True,Gautam A.,"Computer Science Department, Brigham Young Uni...","[Jensen T.D., Error B.M., Raikes A.C., Woods D...","[iwjvd, zquxf, apwsb, asxlp]","[False, False, False, False]"
3,Bridging theory to practice: Feedforward and c...,2020,FullPaper,Graph,True,de Moura Oliveira P.B.,"Department of Chemical Engineering, Brigham Y...","[Bean M., Li X., Chetty V.]","[owxse, kcrvc, xrszk]","[True, True, False]"
6,Development of sediment management guidelines ...,2009,FullPaper,Graph,True,Hajimirzaie S.M.,"Department of Mechanical Engineering, Brigham ...","[Russell D.N., Welling A.G., Gong H., Rosquist...","[nrvsb, wffqz, ptuzm, sbmvm]","[False, True, False, True]"
8,Structural design space exploration using prin...,2009,Poster,NLP,True,Bunnell S.,"School of Communications, Brigham Young Univer...","[Krutka D.G., Cardall H., Stewart E.]","[ryqcf, hoztp, frois]","[True, True, False]"
13,Religion-focused dating apps: A Q methodology ...,2003,Short,NLP,True,Richardson M.,"North Carolina State University, Department of...","[Horak P., Carpenter J.P., Exter M.]","[tovwm, ndcww, gnxsr]","[False, True, False]"


In [186]:
reviewProcess.apply(pd.Series.explode)

,title,year,type,keyword,accepted,person_name,school_name,reviewer_name,comment,decision
0,Self-assessment of Proficiency of Intelligent ...,2015,Poster,Graph,True,Gautam A.,"Computer Science Department, Brigham Young Uni...",Cardall H.,rxnlz,True
0,Self-assessment of Proficiency of Intelligent ...,2015,Poster,Graph,True,Gautam A.,"Computer Science Department, Brigham Young Uni...",Schuh C.A.,vsgap,False
0,Self-assessment of Proficiency of Intelligent ...,2015,Poster,Graph,True,Gautam A.,"Computer Science Department, Brigham Young Uni...",White S.,xtnbj,True
0,Self-assessment of Proficiency of Intelligent ...,2015,Poster,Graph,True,Gautam A.,"Computer Science Department, Brigham Young Uni...",Prestage R.M.,cbmud,True
3,Bridging theory to practice: Feedforward and c...,2020,FullPaper,Graph,True,de Moura Oliveira P.B.,"Department of Chemical Engineering, Brigham Y...",Troyanskaya M.,ahofs,False
...,...,...,...,...,...,...,...,...,...,...
3922,Joining Dissimilar Material Using Friction Sti...,2009,Poster,ML,False,Upadhyay P.,"Department of Physics, Osaka Dental Universit...",Thompson S.K.,tyalf,True
3922,Joining Dissimilar Material Using Friction Sti...,2009,Poster,ML,False,Upadhyay P.,"Department of Physics, Osaka Dental Universit...",Sachdev A.,aepdv,True
3932,A minimal realization technique for the dynami...,2012,FullPaper,NLP,False,Warnick S.,Department of Civil and Environmental Enginee...,Clarke K.D.,qlcqh,False
3932,A minimal realization technique for the dynami...,2012,FullPaper,NLP,False,Warnick S.,Department of Civil and Environmental Enginee...,Krutka D.G.,titrv,True


In [197]:
for a in reviewProcess.itertuples():
    print(a)
    break

Pandas(Index=0, title='Self-assessment of Proficiency of Intelligent Systems: Challenges and Opportunities', year=2015, type='Poster', keyword='Graph', accepted=True, person_name='Gautam A.', school_name='Computer Science Department, Brigham Young University, Provo, UT  84604, United States', reviewer_name=['Jensen T.D.', 'Error B.M.', 'Raikes A.C.', 'Woods D.D.'], comment=['iwjvd', 'zquxf', 'apwsb', 'asxlp'], decision=[False, False, False, False])


In [199]:
for _, article_name, year, _, _, _, _, _, reviewer_name, comment, accepted in reviewProcess.itertuples():
    # get the submission node
    submission_node = URIRef(f"http://kg_sdm.org/Submission/{URLparse(article_title)}")

    # create the reviewProcess node
    review_process = URIRef(f"http://kg_sdm.org/ReviewProcess/{URLparse(author_name+'_'+article_name)}")
    g.add((review_process, RDF.type, KG_SDM.ReviewProcess))
    
    g.add((submission_node, KG_SDM.goes_through, review_process))
    
    # get the reviewer node
    reviewer_node = URIRef(f"http://kg_sdm.org/Person/{URLparse(author_name)}")
#     already added before
#     g.add((reviewer_node, RDF.type, KG_SDM.Reviewer))
    g.add((reviewer_node, KG_SDM.reviews, review_process))
    
    comment_lit = Literal(str(comment))
    
    decision_lit = Literal(bool(accepted))
    
    g.add((review_process, KG_SDM.comment, comment_lit))
    g.add((review_process, KG_SDM.decision, decision_lit))
    


In [200]:
g.serialize()

'@prefix foaf: <http://xmlns.com/foaf/0.1/> .\n@prefix kg_sdm: <http://kg_sdm.org/> .\n@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .\n\n<http://kg_sdm.org/Person/Aarsnes_U_J_> a kg_sdm:Author ;\n    kg_sdm:school " Cardiff University Brain Research Imaging Centre (CUBRIC), Cardiff University, Cardiff, United Kingdom" ;\n    kg_sdm:writes <http://kg_sdm.org/Submission/Creating_open_source_models__test_cases__and_data_for_oilfield_drilling_challenges> ;\n    foaf:name "Aarsnes U.J." .\n\n<http://kg_sdm.org/Person/Abaimov_S_> a kg_sdm:Author ;\n    kg_sdm:school " Department of Biology, Utah Valley University, SB 242M, Orem, UT  84058, United States" ;\n    kg_sdm:writes <http://kg_sdm.org/Submission/In_plane_permeability_characterization_of_engineering_textiles_based_on_radial_flow_experiments__A_benchmark_exercise> ;\n    foaf:name "Abaimov S." .\n\n<http://kg_sdm.org/Person/Abildskov_T_J_> a kg_sdm:Reviewers ;\n    kg_sdm:school " Department of Physics and Astronomy, KU Leuven, Ce

In [201]:
    # get the author node
    
    # create the submission onde
    submission_node = URIRef(f"http://kg_sdm.org/Submission/{URLparse(article_title)}")
    
    # author wrote a paper
    g.add((author_node, KG_SDM.written, submission_node))
                            
                            
    # data for submission
    paper_title_lit = Literal(str(article_title))
    paper_year_lit = Literal(int(year))
    g.add((submission_node, KG_SDM.paper_title, paper_title_lit))
    g.add((submission_node, KG_SDM.paper_year, paper_year_lit))
                            

<Graph identifier=N8fa2603306d840d8ae178634b2ab44a6 (<class 'rdflib.graph.Graph'>)>

# Adding paper submitted in Venue

In [205]:
articles = pd.read_csv('data_kg_sdm/articles.csv')
articles = articles.loc[:len(authors)-1]
articles['accepted'] = ""
articles['accepted'].loc[:500] = True
articles['accepted'].loc[500:] = False
articles.head()

/tmp/ipykernel_40793/1116501579.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  articles['accepted'].loc[:500] = True
/tmp/ipykernel_40793/1116501579.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  articles['accepted'].loc[500:] = False


,ID,title,volume,DOI,accepted
0,1,Self-assessment of Proficiency of Intelligent ...,1210 AISC,10.1007/978-3-030-51758-8_15,True
1,2,Bridging theory to practice: Feedforward and c...,695 LNEE,10.1007/978-3-030-58653-9_3,True
2,3,Development of sediment management guidelines ...,146,10.1061/(ASCE)HY.1943-7900.0001822,True
3,4,Structural design space exploration using prin...,20,10.1115/1.4047428,True
4,5,Religion-focused dating apps: A Q methodology ...,55,10.1016/j.tele.2020.101448,True


In [206]:
publications = pd.read_csv('data_kg_sdm/publications.csv')
publications

,ID,name,Type
0,1,Advances in Intelligent Systems and Computing,Conference Paper
1,2,Lecture Notes in Electrical Engineering,Conference Paper
2,3,Journal of Hydraulic Engineering,Journal
3,4,Journal of Computing and Information Science i...,Journal
4,5,Telematics and Informatics,Conference Paper
...,...,...,...
399,400,CAD Computer Aided Design,Conference Paper
400,401,International Symposium on Advances in Computa...,Conference Paper
401,402,AIAA SPACE and Astronautics Forum and Expositi...,Conference Paper
402,403,IEEE Transactions on Control of Network Systems,Conference Paper


In [207]:


article_publisher_link = pd.read_csv('data_kg_sdm/article_published_by.csv')
article_publisher_link



,article_ID,publisher_ID,year
0,1,433,2020
1,1,435,2019
2,1,780,2019
3,1,612,2020
4,2,378,2020
...,...,...,...
3534,879,141,2016
3535,879,475,2015
3536,880,492,2016
3537,880,300,2017


In [208]:
article_pub = pd.merge(articles, article_publisher_link, left_on='ID', right_on='article_ID')
articles_publishedin = pd.merge(article_pub, publications, left_on='publisher_ID', right_on='ID',how='left')
articles_publishedin


,ID_x,title,volume,DOI,accepted,article_ID,publisher_ID,year,ID_y,name,Type
0,1,Self-assessment of Proficiency of Intelligent ...,1210 AISC,10.1007/978-3-030-51758-8_15,True,1,433,2020,NaN,NaN,NaN
1,1,Self-assessment of Proficiency of Intelligent ...,1210 AISC,10.1007/978-3-030-51758-8_15,True,1,435,2019,NaN,NaN,NaN
2,1,Self-assessment of Proficiency of Intelligent ...,1210 AISC,10.1007/978-3-030-51758-8_15,True,1,780,2019,NaN,NaN,NaN
3,1,Self-assessment of Proficiency of Intelligent ...,1210 AISC,10.1007/978-3-030-51758-8_15,True,1,612,2020,NaN,NaN,NaN
4,2,Bridging theory to practice: Feedforward and c...,695 LNEE,10.1007/978-3-030-58653-9_3,True,2,378,2020,378.0,ICCM International Conferences on Composite Ma...,Conference Paper
...,...,...,...,...,...,...,...,...,...,...,...
3534,879,A minimal realization technique for the dynami...,4,10.1109/TCNS.2015.2498468,False,879,141,2016,141.0,Metallurgical and Materials Transactions A: Ph...,Conference Paper
3535,879,A minimal realization technique for the dynami...,4,10.1109/TCNS.2015.2498468,False,879,475,2015,NaN,NaN,NaN
3536,880,Dryland agriculture in north america,NaN,10.1007/978-3-319-47928-6_15,False,880,492,2016,NaN,NaN,NaN
3537,880,Dryland agriculture in north america,NaN,10.1007/978-3-319-47928-6_15,False,880,300,2017,300.0,IEEE Reviews in Biomedical Engineering,Conference Paper


In [209]:
total = len(articles_publishedin)
nb_joun = int(total * 0.2)
nb_workshop = int(total * 0.3)
nb_symposium = int(total * 0.5)


In [210]:


journals = articles_publishedin.loc[:nb_joun]
workshops = articles_publishedin.loc[nb_joun : nb_joun + nb_workshop]
symposium = articles_publishedin.loc[nb_joun + nb_workshop : ]



In [211]:
journals.head()

,ID_x,title,volume,DOI,accepted,article_ID,publisher_ID,year,ID_y,name,Type
0,1,Self-assessment of Proficiency of Intelligent ...,1210 AISC,10.1007/978-3-030-51758-8_15,True,1,433,2020,NaN,NaN,NaN
1,1,Self-assessment of Proficiency of Intelligent ...,1210 AISC,10.1007/978-3-030-51758-8_15,True,1,435,2019,NaN,NaN,NaN
2,1,Self-assessment of Proficiency of Intelligent ...,1210 AISC,10.1007/978-3-030-51758-8_15,True,1,780,2019,NaN,NaN,NaN
3,1,Self-assessment of Proficiency of Intelligent ...,1210 AISC,10.1007/978-3-030-51758-8_15,True,1,612,2020,NaN,NaN,NaN
4,2,Bridging theory to practice: Feedforward and c...,695 LNEE,10.1007/978-3-030-58653-9_3,True,2,378,2020,378.0,ICCM International Conferences on Composite Ma...,Conference Paper


In [212]:


venue_type_list = [journals,workshops,symposium]
venue_sub_type_ls = ["Journal","Workshop","Symposium"]



In [216]:


for i,venue_tp in enumerate(venue_type_list):

    venue_type = venue_sub_type_ls[i]
    venue_sub_type = URIRef(f"http://kg_sdm.org/{venue_type}")
    venue_tp.dropna(inplace=True)
    
    for index,row in venue_tp.iterrows():
        year = row['year']
        year_literal = Literal(int(year))
        submissiontitle = row['title']
        confname = row['name']
        decision = row['accepted']
        
        # parsing conference
        conf_title = URLparse(confname)
        conf_node = URIRef(f"http://kg_sdm.org/venue/{conf_title}")
        venue_lit = Literal(str(conf_title))

        # parsing submission
        sub_title = URLparse(submissiontitle)
        sub_node = URIRef(f"http://kg_sdm.org/Submission/{sub_title}")

        
        # parsing for publication
        # assuming only half of the articles got accepted
        if decision:
            if venue_type=='Journal':
                Pub_title = conf_title+'_volume_'+str(random.randint(1, 5))
                Pub_title_lit = Literal(str(Pub_title))
                
                pub_node = URIRef(f"http://kg_sdm.org/Publication/{Pub_title}")
                g.add((pub_node, RDF.type, KG_SDM.Volume))
                g.add((sub_node,KG_SDM.published_in,pub_node))
                g.add((pub_node, KG_SDM.publication_title,Pub_title_lit))
                g.add((pub_node, KG_SDM.publication_year,year_literal))

            # if (venue_type=='Workshop' or venue_type=='Symposium')
            else:
                Pub_title = conf_title+'_proceddings_'
                Pub_title_lit = Literal(str(Pub_title))
                
                pub_node = URIRef(f"http://kg_sdm.org/Publication/{Pub_title}")
                g.add((pub_node, RDF.type, KG_SDM.Proceddings))
                g.add((sub_node,KG_SDM.published_in,pub_node))
                g.add((pub_node, KG_SDM.publication_title,Pub_title_lit))
                g.add((pub_node, KG_SDM.publication_year,year_literal))



        # add subclass type
        g.add((conf_node, RDF.type, venue_sub_type))
        # add name of venue
        g.add((conf_node, KG_SDM.venue_title, venue_lit))

        # connect conference and submission
        g.add((sub_node,KG_SDM.submitted_to,conf_node))



/tmp/ipykernel_40793/2809025766.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  venue_tp.dropna(inplace=True)
